In [7]:
import tensorflow as tf
from tensorflow import keras

DATA_PATH = 'v2_cam1_cam2_ split_by_driver/Camera 2/test'
NUM_CLASSES = 10
BATCH_SIZE = 64

In [8]:
import glob

images_path, class_ids = [], []
for image_path in glob.glob(DATA_PATH + '/*/*'):
    class_id = image_path.split('/')[-2][-1]
    images_path.append(image_path)
    class_ids.append(class_id)

In [14]:
def preprocess(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = keras.applications.mobilenet.preprocess_input(img)
    print(img)
    return img, int(label)

In [15]:
data = tf.data.Dataset.from_tensor_slices((images_path, class_ids))

In [16]:
data = data.map(preprocess, num_parallel_calls=6).batch(BATCH_SIZE).prefetch(1)

Tensor("sub:0", shape=(224, 224, 3), dtype=float32)


In [17]:
model = keras.Sequential([
    keras.applications.MobileNet(input_shape=(224, 224, 3), include_top=False),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(NUM_CLASSES)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 10)                10250     
                                                                 
Total params: 3,239,114
Trainable params: 3,217,226
Non-trainable params: 21,888
_________________________________________________________________


In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.evaluate(data)

13/13 [==============================] - 10s 712ms/step - loss: 9.9375 - accuracy: 0.0763


[9.937511444091797, 0.07625000178813934]